In [13]:
!pip install diffusers transformers gradio accelerate pillow

In [18]:
# 1-2. DATASET CREATION (ALREADY WORKING - SKIP IF FOLDERS EXIST)
print("📁 Checking dataset...")
if not os.path.exists("dataset/damaged_noisy/photo_00.jpg"):
    print("🔄 Creating dataset...")
    photo_dir = "old-photos"
    image_files = [f for f in os.listdir(photo_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    dataset = [Image.open(os.path.join(photo_dir, f)).convert('RGB') for f in image_files]
    
    os.makedirs("dataset/clean", exist_ok=True)
    os.makedirs("dataset/damaged_noisy", exist_ok=True)
    
    def generate_damages(clean_img, idx):
        clean_img.save(f"dataset/clean/photo_{idx:02d}.jpg")
        noisy = np.array(clean_img) + np.random.normal(0, 25, np.array(clean_img).shape)
        noisy_img = Image.fromarray(np.clip(noisy, 0, 255).astype(np.uint8))
        noisy_img.save(f"dataset/damaged_noisy/photo_{idx:02d}.jpg")
    
    for i, img in enumerate(dataset):
        generate_damages(img, i)
    print(f"✅ Dataset created: {len(dataset)} images")
else:
    print("✅ Dataset already exists!")

# 3. FIXED MODEL LOAD (NO float16 + CPU SAFE)
print("🤖 Loading Model...")
pipe = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# 4. FIXED DENOISE FUNCTION (3 LINES CHANGED!)
def denoise_image(noisy_image):
    # Ensure PIL format
    if isinstance(noisy_image, np.ndarray):
        noisy_image = Image.fromarray(noisy_image)
    noisy_image = noisy_image.convert('RGB')
    
    # Create proper mask (full white = denoise entire image)
    mask = Image.new('L', noisy_image.size, 255)
    
    # FIXED CALL - All parameters correct!
    result = pipe(
        prompt="high quality restored old photo, clear, sharp, detailed",
        image=noisy_image,
        mask_image=mask,
        strength=0.8
    ).images[0]
    return result

# 5. TEST (WORKS 100%!)
print("🧪 Testing...")
sample_noisy = Image.open("dataset/damaged_noisy/photo_00.jpg")
result = denoise_image(sample_noisy)
result.save("day1_denoised_result.png")
print("✅ SAVED: day1_denoised_result.png")

# 6. GRADIO UI
demo = gr.Interface(
    fn=denoise_image,
    inputs=gr.Image(type="pil", label="Upload Noisy Old Photo"),
    outputs=gr.Image(type="pil", label="✨ Restored Result"),
    title="🔥Old Photo Denoising",
    description="29 Real Old Photos Restored!"
)
demo.launch(share=True)

📁 Checking dataset...
✅ Dataset already exists!
🤖 Loading Model...


Loading pipeline components...:  57%|██████████████████████████████▎                      | 4/7 [00:00<00:00, 20.80it/s]An error occurred while trying to fetch /home/dodo/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/dodo/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/dodo/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/dodo/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to u

🧪 Testing...


100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [08:28<00:00, 12.72s/it]


✅ SAVED: day1_denoised_result.png
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c5ee2cdcc36e128128.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
